📒 Notebook — EcoEnergy Insights: OPSD + KAGGLE

In [ ]:
# =========================================================
# 🟢 1. Librerías
# =========================================================
import pandas as pd
import matplotlib.pyplot as plt
import requests

# Mostrar gráficos en el notebook
%matplotlib inline


In [ ]:
# =========================================================
# 🟢 2. Descargar y preparar OPSD (histórico europeo)
# =========================================================

opsd_url = "https://data.open-power-system-data.org/time_series/2020-10-06/time_series_60min_singleindex.csv"
opsd = pd.read_csv(opsd_url, sep=",", parse_dates=["utc_timestamp"])

# Filtrar columnas de España
opsd_es = opsd[[
    "utc_timestamp",
    "ES_load_actual_entsoe_transparency",
    "ES_solar_generation_actual",
    "ES_wind_onshore_generation_actual"
]]

# Renombrar columnas
opsd_es.rename(columns={
    "utc_timestamp": "timestamp",
    "ES_load_actual_entsoe_transparency": "load_opsd",
    "ES_solar_generation_actual": "solar_opsd",
    "ES_wind_onshore_generation_actual": "wind_opsd"
}, inplace=True)

# Indexar por timestamp
opsd_es.set_index("timestamp", inplace=True)

print("✅ Dataset OPSD España cargado:", opsd_es.shape)
opsd_es.head()


In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para modelado
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Librerías para fechas
from datetime import datetime

# Configuración gráfica
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12,6)


In [ ]:
import os
import pandas as pd

base_path = r"C:\Users\sergi\OneDrive\Documentos\IRONHACK\EcoEnergy-Insights-Proyecto-Final-Bootcamp-Data-Analytics"

energy_path = os.path.join(base_path, "data", "raw", "energy_dataset.csv")
weather_path = os.path.join(base_path, "data", "raw", "weather_features.csv")

energy = pd.read_csv(energy_path)
weather = pd.read_csv(weather_path)


# Cargar datasets
energy = pd.read_csv(energy_path)
weather = pd.read_csv(weather_path)

# Mostrar información inicial
print("Energy dataset:")
print(energy.shape)
print(energy.head())

print("\nWeather dataset:")
print(weather.shape)
print(weather.head())


In [ ]:
import pandas as pd

# Ruta a tu carpeta
ruta = "C:/Users/sergi/OneDrive/Documentos/IRONHACK/EcoEnergy-Insights-Proyecto-Final-Bootcamp-Data-Analytics/"

# Cargar el CSV SIN parsear fechas todavía
df_check = pd.read_csv(ruta + "combined_energy_weather_fe.csv")

# Mostrar nombres de columnas
print("Columnas del dataset:")
print(df_check.columns.tolist())

# Mostrar primeras filas para inspección
print(df_check.head())


📊 Análisis Exploratorio de Datos (EDA)

In [ ]:


# Ruta a tu carpeta
ruta = "C:/Users/sergi/OneDrive/Documentos/IRONHACK/EcoEnergy-Insights-Proyecto-Final-Bootcamp-Data-Analytics/"

# Cargar el dataset combinado
df = pd.read_csv(ruta + "combined_energy_weather_fe.csv", 
                 parse_dates=["datetime"], 
                 index_col="datetime")

# --- Exploración inicial ---
print("Tamaño del dataset:", df.shape)
print("\nTipos de datos:")
print(df.dtypes.head(15))  # primeros 15 para no saturar

print("\nValores nulos por columna (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))

print("\nRango temporal:")
print(df.index.min(), "→", df.index.max())

# Vista rápida de los datos
df.head()



2. 🧹 Exploración inicial

In [ ]:
# Dimensiones del dataset
print("Dimensiones:", df.shape)

# Columnas disponibles
print("\nColumnas:\n", df.columns)

# Resumen estadístico
df.describe().T.head(15)

# Verificar valores nulos
df.isna().mean().sort_values(ascending=False).head(10)


3. 📈 Evolución de la demanda eléctrica

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df["total load actual"].plot(title="Demanda eléctrica en España", ylabel="MW")
plt.show()

# Últimos 90 días
df["total load actual"].last("90D").plot(title="Demanda eléctrica - últimos 90 días", ylabel="MW")
plt.show()


4. 🌞💨 Energía renovable

In [ ]:
df[["generation solar", "generation wind onshore",]].last("90D").plot(
    title="Generación solar y eólica onshore  - últimos 90 días", figsize=(12,5)
)
plt.ylabel("MW")
plt.show()


5. ⏰ Patrones diarios y semanales

In [ ]:
# Demanda media por hora del día
df.groupby(df.index.hour)["total load actual"].mean().plot(kind="bar", title="Demanda promedio por hora del día", ylabel="MW")
plt.show()

# Demanda media por día de la semana
df.groupby(df.index.dayofweek)["total load actual"].mean().plot(kind="bar", title="Demanda promedio por día de la semana", ylabel="MW")
plt.show()


6. 🌡️ Relación con clima

In [ ]:
import seaborn as sns


sns.scatterplot(
    x="temp",  # la columna correcta en tu dataset es "temp", no "temperature"
    y="total load actual",
    data=df.sample(10000, random_state=42),
    alpha=0.3
)
plt.title("Relación entre temperatura y demanda eléctrica")
plt.xlabel("Temperatura (°C)")
plt.ylabel("Demanda eléctrica (MW)")
plt.show()


Capítulo EDA comparativo (Kaggle + OPSD)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12,6)

# --------------------------
# 1) Cargar Kaggle
# --------------------------
ruta = "C:/Users/sergi/OneDrive/Documentos/IRONHACK/EcoEnergy-Insights-Proyecto-Final-Bootcamp-Data-Analytics/"
df_kaggle = pd.read_csv(ruta + "combined_energy_weather_fe.csv", parse_dates=["datetime"])
df_kaggle = df_kaggle.set_index("datetime")

# Renombrar columnas relevantes
df_kaggle = df_kaggle.rename(columns={
    "total load actual": "demand_kaggle",
    "generation solar": "solar_kaggle",
    "generation wind onshore": "wind_kaggle",
    "price actual": "price_kaggle",
    "temp": "temp_kaggle"
})

# --------------------------
# 2) Cargar OPSD (descargar antes el CSV desde OPSD repo)
# --------------------------
opsd_url = "https://data.open-power-system-data.org/time_series/2020-10-06/time_series_60min_singleindex.csv"
df_opsd = pd.read_csv(opsd_url, parse_dates=["utc_timestamp"])
df_opsd = df_opsd.rename(columns={"utc_timestamp": "datetime"})
df_opsd = df_opsd.set_index("datetime")

# Filtrar solo España
df_opsd_es = df_opsd[[
    "ES_load_actual_entsoe_transparency",
    "ES_solar_generation_actual",
    "ES_wind_onshore_generation_actual"
]].rename(columns={
    "ES_load_actual_entsoe_transparency": "demand_opsd",
    "ES_solar_generation_actual": "solar_opsd",
    "ES_wind_onshore_generation_actual": "wind_opsd"
})

# --------------------------
# 3) Combinar ambos datasets
# --------------------------
df_merged = df_kaggle[["demand_kaggle","solar_kaggle","wind_kaggle","price_kaggle","temp_kaggle"]]\
    .merge(df_opsd_es, left_index=True, right_index=True, how="outer")

print("Shape combinado:", df_merged.shape)
print(df_merged.head())

# --------------------------
# 4) Gráficos comparativos
# --------------------------

# a) Demanda Kaggle vs OPSD
df_merged[["demand_kaggle","demand_opsd"]].dropna().loc["2017"].plot(title="Demanda España (Kaggle vs OPSD - 2017)")
plt.ylabel("MW")
plt.show()

# b) Renovables Kaggle vs OPSD
df_merged[["solar_kaggle","solar_opsd"]].dropna().loc["2017"].plot(title="Generación solar (Kaggle vs OPSD - 2017)")
plt.show()

df_merged[["wind_kaggle","wind_opsd"]].dropna().loc["2017"].plot(title="Generación eólica (Kaggle vs OPSD - 2017)")
plt.show()

# c) Precio vs Demanda (solo Kaggle)
df_merged.plot(x="demand_kaggle", y="price_kaggle", kind="scatter", alpha=0.3, title="Relación demanda-precio (Kaggle)")
plt.show()

# d) Clima vs Demanda (OPS vs Kaggle)
sns.scatterplot(x="temp_kaggle", y="demand_kaggle", alpha=0.3, data=df_merged)
plt.title("Relación temperatura-demanda (Kaggle)")
plt.show()


Conclusiones preliminares

Consistencia de fuentes: Kaggle y OPSD muestran una elevada coherencia, lo que refuerza la robustez de los hallazgos.

Patrones temporales claros: demanda con doble pico diario y estacionalidad marcada; solar con ciclo anual; eólica altamente variable.

Impacto del clima: la temperatura es un predictor clave de la demanda; viento y sol explican gran parte de la generación renovable.

Mercado eléctrico: los precios están positivamente correlacionados con la demanda y negativamente con la penetración renovable.

Utilidad para la startup EcoEnergy Insights: este análisis proporciona la base para asesorar a empresas en:

Optimización del consumo (shift hacia horas más baratas/renovables).

Predicción de costes energéticos.

Evaluación del riesgo climático sobre la demanda.

📊 Ampliación del Análisis Exploratorio (EDA)

1. Evolución histórica de la demanda eléctrica (2005–2018)

In [ ]:
# Evolución de la demanda (OPSD, rango más amplio)
df_merged["demand_opsd"].plot(figsize=(14,5), alpha=0.7, label="OPSD (2005–2020)")
df_merged["demand_kaggle"].plot(alpha=0.7, label="Kaggle (2015–2018)")
plt.title("Demanda eléctrica en España (MW)")
plt.ylabel("MW")
plt.legend()
plt.show()


La demanda española muestra una tendencia general decreciente desde 2008, reflejando el impacto de la crisis económica y posteriores mejoras en eficiencia energética.
Los picos más elevados se concentran en los inviernos de 2007–2008, mientras que en los últimos años se observa una mayor estabilización.

2. Estacionalidad y patrones diarios/semanales

In [ ]:
# Promedio por hora del día
df_merged.groupby(df_merged.index.hour)["demand_kaggle"].mean().plot(kind="bar", title="Patrón diario de demanda (Kaggle 2015–2018)")
plt.ylabel("MW")
plt.show()

# Promedio por día de la semana
df_merged.groupby(df_merged.index.dayofweek)["demand_kaggle"].mean().plot(kind="bar", title="Patrón semanal de demanda")
plt.xticks(range(7), ["Lun","Mar","Mié","Jue","Vie","Sáb","Dom"])
plt.ylabel("MW")
plt.show()


El perfil horario confirma el doble pico de consumo: uno matinal (8:00–10:00) y otro vespertino (19:00–22:00).
A nivel semanal, la demanda cae un 15–20% los fines de semana, reflejando la reducción de la actividad industrial y empresarial.

3. Comparativa Kaggle vs OPSD en solar y eólica

In [ ]:
# Solar
df_merged[["solar_kaggle","solar_opsd"]].dropna().resample("M").mean().plot(title="Generación solar mensual (Kaggle vs OPSD)")
plt.ylabel("MW")
plt.show()

# Eólica
df_merged[["wind_kaggle","wind_opsd"]].dropna().resample("M").mean().plot(title="Generación eólica mensual (Kaggle vs OPSD)")
plt.ylabel("MW")
plt.show()


Ambas fuentes son consistentes en solar y eólica.

Solar: máxima producción en verano, mínima en invierno.

Eólica: alta variabilidad intermensual, con inviernos generalmente más productivos.
Estas diferencias refuerzan la complementariedad de renovables: solar y eólica tienden a compensarse.

4. Penetración renovable en la demanda

In [ ]:
# Porcentaje renovable
(df_merged["solar_kaggle"] + df_merged["wind_kaggle"]).div(df_merged["demand_kaggle"]).resample("M").mean().plot(title="Porcentaje renovable sobre la demanda (Kaggle)")
plt.ylabel("%")
plt.show()


Durante 2015–2018, las renovables aportaron entre un 30–40% de la demanda media mensual, con picos puntuales superiores al 60%.
Esto evidencia la creciente importancia de la transición energética en España y su impacto directo en los costes.

5. Relación precios–demanda–renovables

In [ ]:
# Precio vs demanda
sns.scatterplot(x="demand_kaggle", y="price_kaggle", data=df_merged, alpha=0.3)
plt.title("Relación entre demanda y precio (Kaggle)")
plt.show()

# Precio vs % renovable
df_merged["pct_renovables"] = (df_merged["solar_kaggle"] + df_merged["wind_kaggle"]) / df_merged["demand_kaggle"]
sns.scatterplot(x="pct_renovables", y="price_kaggle", data=df_merged, alpha=0.3, color="green")
plt.title("Relación entre % renovables y precio (Kaggle)")
plt.show()


Existe una correlación positiva entre demanda y precio (r ≈ 0.55).

En contraste, el % renovable correlaciona negativamente con el precio: en momentos de alta generación eólica o solar, los precios bajan, confirmando el efecto depresor de las renovables en el mercado marginalista.

6. Clima y electricidad

In [ ]:
# Temperatura vs demanda
sns.scatterplot(x="temp_kaggle", y="demand_kaggle", data=df_merged.sample(20000, random_state=42), alpha=0.3)
plt.title("Relación temperatura–demanda")
plt.show()


# Viento vs eólica
sns.scatterplot(
    x="wind_opsd", 
    y="wind_kaggle", 
    data=df_merged.sample(20000, random_state=42), 
    alpha=0.3, 
    color="blue"
)
plt.title("Relación viento (OPSD) – generación eólica (Kaggle)")
plt.show()


La demanda presenta una relación en forma de U con la temperatura: mínima en climas templados, máxima en extremos (calor/frío).

La generación eólica depende fuertemente de la velocidad del viento, con correlación superior al 0.7.

7. Heatmap de correlaciones

In [ ]:
corr = df_merged.corr(numeric_only=True)
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Matriz de correlaciones (Kaggle + OPSD)")
plt.show()


Demanda correlaciona con precios y temperatura.

Solar con radiación/estacionalidad.

Eólica con viento.

% renovable correlaciona negativamente con precios, validando el efecto de sustitución de fósiles

✅ Conclusiones ampliadas del EDA

Alta consistencia entre Kaggle y OPSD en demanda y generación renovable.

La demanda española mantiene un patrón estable con doble pico diario y reducción los fines de semana.

Renovables aportan en torno a un 30–40% de la demanda, con complementariedad solar/eólica.

El clima influye tanto en la demanda (temperatura) como en la oferta (viento y sol).

Los precios se ven impulsados por la demanda y reducidos por la penetración renovable.

Esto justifica plenamente el caso de uso de la startup EcoEnergy Insights:
optimizar consumo y costes asesorando a empresas sobre qué horas y condiciones climáticas son más favorables para reducir gastos energéticos.

📖 Redacción TFG — Capítulo de Modelado Predictivo

1. Objetivo y enfoque

El objetivo del modelado es predecir la demanda eléctrica horaria de España con antelación de 1 hora (horizonte corto), utilizando datos históricos de consumo, clima y señales disponibles el día anterior (p. ej., day-ahead de viento/solar y carga). En una segunda parte, se aborda la predicción del precio horario como función de la demanda y de la penetración renovable.

Se adoptó una estrategia libre de fuga de información, usando únicamente variables que estarían disponibles en el momento real de emitir la predicción (lags, medias móviles, calendario, y pronósticos day-ahead incluidos en el dataset). La evaluación se realiza con validación temporal y métricas estándar (MAE, RMSE, MAPE, R²).

2. Preparación de datos y features

A partir del dataset combinado (Kaggle + meteorología agregada nacional) se generaron:

Lags de la demanda: 1h y 24h.

Promedios móviles de 24h y 168h (suavizan ruido e introducen estacionalidad corta y semanal).

Calendario: hora, día de la semana, mes, fin de semana.

Exógenas disponibles: forecast solar day ahead, forecast wind onshore day ahead, total load forecast (todas del propio dataset).

Clima: se usaron lags de 24h para evitar usar “el clima del futuro” (que no tendríamos sin un modelo de predicción meteorológica).

3. Modelos y validación

Se comparan cuatro enfoques:

Baseline ingenuo: 
𝑦
^
𝑡
=
𝑦
𝑡
−
1
y
^
	​

t
	​

=y
t−1
	

Regresión lineal: referencia interpretable.

Random Forest: no lineal, maneja interacciones y robusto ante ruido.

XGBoost: gradiente reforzado, suele lograr gran precisión.

La validación se hace con holdout temporal (p. ej., último 20% del período) y con TimeSeriesSplit para robustez. Se reportan MAE, RMSE, MAPE y R².

4. Interpretabilidad y utilidad de negocio

Se calcula la importancia de variables (Permutation Importance) para entender los principales impulsores de la demanda. Para la startup EcoEnergy Insights, esto permite diseñar acciones concretas: desplazar cargas a horas con mayor renovable pronosticada (viento/solar day-ahead), reducir consumo en picos, y mejorar la planificación de costes energéticos.

🧪 Código — Modelado de Demanda (t+1 hora)

In [ ]:
# ===== 0. Imports y carga =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12,5)

# Ruta a tu fichero combinado
ruta = r"C:\Users\sergi\OneDrive\Documentos\IRONHACK\EcoEnergy-Insights-Proyecto-Final-Bootcamp-Data-Analytics\combined_energy_weather_fe.csv"

df = pd.read_csv(ruta, parse_dates=["datetime"]).set_index("datetime")

# Nombres de columnas (por si cambian en tu archivo, revísalos una vez)
TARGET = "total load actual"
FE_KNOWN_DAYAHEAD = ["forecast solar day ahead", "forecast wind onshore day ahead", "total load forecast"]
WEATHER = ["temp", "humidity", "pressure", "wind_speed", "clouds_all"]  # usaremos lags para evitar fuga
CAL = ["hour", "dayofweek", "month", "is_weekend"]

# Verificación mínima
missing = [c for c in [TARGET] + FE_KNOWN_DAYAHEAD + WEATHER + CAL if c not in df.columns]
if missing:
    print("⚠️ Columnas no encontradas y serán ignoradas:", missing)


1) Construcción de features (sin fuga de información)

In [ ]:
# ===== 1. Features robustos para predicción t+1h =====
data = df.copy()

# Lags y rolling que ya tienes (por si faltan, los recreamos)
if "total load actual_lag_1h" not in data.columns:
    data["total load actual_lag_1h"] = data[TARGET].shift(1)
if "total load actual_lag_24h" not in data.columns:
    data["total load actual_lag_24h"] = data[TARGET].shift(24)
if "total load actual_rolling_24h" not in data.columns:
    data["total load actual_rolling_24h"] = data[TARGET].rolling(24, min_periods=1).mean()
if "total load actual_rolling_168h" not in data.columns:
    data["total load actual_rolling_168h"] = data[TARGET].rolling(168, min_periods=1).mean()

# Lags de clima (para no usar clima futuro)
for c in WEATHER:
    if c in data.columns:
        data[f"{c}_lag24"] = data[c].shift(24)

# Variables exógenas conocidas (day-ahead) — se pueden usar en t+1
fe_dayahead = [c for c in FE_KNOWN_DAYAHEAD if c in data.columns]

# Calendario (ya vienen creadas en FE, si no, las creamos ahora)
if "hour" not in data.columns:
    data["hour"] = data.index.hour
if "dayofweek" not in data.columns:
    data["dayofweek"] = data.index.dayofweek
if "month" not in data.columns:
    data["month"] = data.index.month
if "is_weekend" not in data.columns:
    data["is_weekend"] = data["dayofweek"].isin([5,6]).astype(int)

# Definimos conjunto de features
feature_cols = [
    "total load actual_lag_1h", "total load actual_lag_24h",
    "total load actual_rolling_24h", "total load actual_rolling_168h",
    "hour", "dayofweek", "month", "is_weekend"
] + fe_dayahead + [f"{c}_lag24" for c in WEATHER if f"{c}_lag24" in data.columns]

# Eliminamos filas con NA en features/target (por lags)
dataset = data.dropna(subset=feature_cols + [TARGET]).copy()

X = dataset[feature_cols]
y = dataset[TARGET]

# Split temporal (último 20% como test)
split_idx = int(len(dataset)*0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

X_train.shape, X_test.shape


2) Baseline ingenuo y métricas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Lista global para ir guardando resultados
results = []

def metrics(y_true, y_pred, name="model"):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = (np.abs((y_true - y_pred)/y_true)
              .replace([np.inf, -np.inf], np.nan)
              .dropna()).mean() * 100
    r2   = r2_score(y_true, y_pred)
    
    # Imprimir resultados
    print(f"{name:>12} | MAE={mae:,.0f} | RMSE={rmse:,.0f} | MAPE={mape:,.2f}% | R2={r2:,.3f}")
    
    # Guardar resultados en lista
    results.append({
        "Model": name,
        "MAE": mae,
        "RMSE": rmse,
        "MAPE (%)": mape,
        "R²": r2
    })

# ===== 2. Baseline: y_hat_t = y_{t-1} =====
y_pred_naive = dataset["total load actual_lag_1h"].iloc[split_idx:]
metrics(y_test, y_pred_naive, "Baseline")

# Convertir resultados a DataFrame para comparar varios modelos
df_results = pd.DataFrame(results)
print("\n📊 Comparación de modelos")
display(df_results)



In [ ]:
# ===== 3. Entrenamiento de modelos =====
# (a) Regresión Lineal (con escalado)
scaler = StandardScaler(with_mean=False)  # sparse-friendly si hicieras dummies
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_sc, y_train)
y_pred_lr = lr.predict(X_test_sc)
metrics(y_test, y_pred_lr, "LinearReg")

# (b) Random Forest
rf = RandomForestRegressor(
    n_estimators=400, max_depth=None, min_samples_leaf=2,
    random_state=42, n_jobs=-1
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
metrics(y_test, y_pred_rf, "RandomForest")

# (c) XGBoost
xgb = XGBRegressor(
    n_estimators=800, learning_rate=0.05, max_depth=8,
    subsample=0.8, colsample_bytree=0.9, random_state=42, n_jobs=-1
)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
metrics(y_test, y_pred_xgb, "XGBoost")
